# 🔀 Question Classifier: The Routing Pattern

Welcome to routing! This pattern shines when:
- Different inputs need specialized handling
- Classification can be done accurately
- Optimizing for one type might hurt others

Perfect for directing OpenSearch questions to the right experts! 🎯

In [ ]:
import boto3

# No need to reinvent the wheel, lets pull this from our utilities folder
from utils.retrieval_utils import get_chroma_os_docs_collection, ChromaDBRetrievalClient

# Initialize the Bedrock client
REGION = 'us-west-2'
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name=REGION)

# We've pushed the retrieval client from the prompt chaining notebook to the retrieval utils for simplicity
chroma_os_docs_collection: ChromaDBRetrievalClient = get_chroma_os_docs_collection()

print("✅ Client setup and retrieval client complete!")

print(len(chroma_os_docs_collection.retrieve(query_text="How do I install OpenSearch on AWS?", n_results=1)))

# Setup Bedrock Helpers
Next we'll reuse the helpers from the prompt chaining lab here

In [ ]:
from typing import Type, Dict, Any, List

# We pushed the base propmt from the previous lab to a a base prompt file.
from utils.base_prompt import BasePrompt
from utils.retrieval_utils import RetrievalResult

def call_bedrock(prompt: BasePrompt) -> str:
    kwargs = {
        "modelId": prompt.model_id,
        "inferenceConfig": prompt.hyperparams,
        "messages": prompt.to_bedrock_messages(),
        "system": prompt.to_bedrock_system(),
    }

    # Call Bedrock
    converse_response: Dict[str, Any] = bedrock.converse(**kwargs)
    # Get the model's text response
    return converse_response['output']['message']['content'][0]['text']

# Helper function to call bedrock
def do_rag(user_input: str, rag_prompt: Type[BasePrompt]) -> str:
    # Retrieve the context from the vector store
    retrieval_results: List[RetrievalResult] = chroma_os_docs_collection.retrieve(user_input, n_results=2)
    # Format the context into a string
    context: str = "\n\n".join([result.document for result in retrieval_results])

    print("Retrieval done")
    # Create the RAG prompt
    inputs: Dict[str, Any] = {"question": user_input, "context": context}
    rag_prompt: BasePrompt = rag_prompt(inputs=inputs)
    # Call Bedrock with the RAG prompt

    print("Calling Bedrock")
    return call_bedrock(rag_prompt)

## 1. Creating Our Question Router

We'll build a classifier that routes questions to specialized handlers for:
- Installation & Setup
- Security & Authentication
- Querying & Indexing
- Performance Optimization

First lets define our prompts

In [ ]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Dict, Any, List

# Define the system prompt
CLASSIFY_SYSTEM_PROMPT = """
You are a helpful assistant specializing in OpenSearch documentation and support.
"""

RAG_SYSTEM_PROMPT = """
You are a helpful assistant specializing in OpenSearch documentation and support.
<instructions>
1. Answer the question using only the documentation provided
2. Be clear and concise with your answer. Your answers should be short, direct, and to the point
3. Avoid saying "based on the context provided"
4. If the answer isn't in the documentation, say "I don't know".
</instructions>
"""

# Define reusable prompt templates as constants
CLASSIFY_PROMPT_TEMPLATE = """
Classify this OpenSearch question into exactly one category:

Question: {question}

Categories:
- INSTALL: Installation, setup, cluster configuration
- SECURITY: Security, authentication, access control
- QUERY: Querying, indexing, search operations
- PERFORMANCE: Optimization, scaling, monitoring

Respond with only the category code (e.g., 'INSTALL').
"""

INSTALLATION_PROMPT_TEMPLATE = """
Using the users qusetions below and provided context, provide detailed installation and setup guidance for OpenSearch:

<question>
{question}
</question>

<context>
{context}
</context>

Include:
- Step-by-step instructions
- System requirements
- Configuration options
- Common issues and solutions

Remember, your answers should be short, direct, and to the point. DO NOT include any preamble or introduction and DO NOT say "based on the context provided" or anything similar.
"""

SECURITY_PROMPT_TEMPLATE = """
Using the users qusetions below and provided context, provide security guidance for OpenSearch:

<question>
{question}
</question>

<context>
{context}
</context>

Include:
- Security best practices
- Authentication setup
- Access control configuration
- Security implications

Remember, your answers should be short, direct, and to the point. DO NOT include any preamble or introduction and DO NOT say "based on the context provided" or anything similar.

"""

QUERY_PROMPT_TEMPLATE = """
Using the users qusetions below and provided context, provide guidance on OpenSearch querying and indexing:

<question>
{question}
</question>

<context>
{context}
</context>

Include:
- Query examples
- Index configuration
- Best practices
- Performance considerations

Remember, your answers should be short, direct, and to the point. DO NOT include any preamble or introduction and DO NOT say "based on the context provided" or anything similar.

"""

PERFORMANCE_PROMPT_TEMPLATE = """
Using the users qusetions below and provided context, provide performance optimization guidance for OpenSearch:

<question>
{question}
</question>

<context>
{context}
</context>

Include:
- Optimization strategies
- Scaling considerations
- Monitoring tips
- Resource management

Remember, your answers should be short, direct, and to the point. DO NOT include any preamble or introduction and DO NOT say "based on the context provided" or anything similar.
"""

# Define prompt classes that inherit from BasePrompt
class ClassifyPrompt(BasePrompt):
    system_prompt: str = CLASSIFY_SYSTEM_PROMPT
    user_prompt: str = CLASSIFY_PROMPT_TEMPLATE

class InstallationPrompt(BasePrompt):
    system_prompt: str = RAG_SYSTEM_PROMPT
    user_prompt: str = INSTALLATION_PROMPT_TEMPLATE

class SecurityPrompt(BasePrompt):
    system_prompt: str = RAG_SYSTEM_PROMPT
    user_prompt: str = SECURITY_PROMPT_TEMPLATE

class QueryPrompt(BasePrompt):
    system_prompt: str = RAG_SYSTEM_PROMPT
    user_prompt: str = QUERY_PROMPT_TEMPLATE

class PerformancePrompt(BasePrompt):
    system_prompt: str = RAG_SYSTEM_PROMPT
    user_prompt: str = PERFORMANCE_PROMPT_TEMPLATE

# Define the WorkflowState using TypedDict
class WorkflowState(TypedDict):
    question: str
    category: str
    response: str

Next lets define our nodes

In [ ]:
def classify_question(state: WorkflowState) -> Dict[str, str]:
    """Classifies the question into a category"""
    inputs = {"question": state['question']}
    prompt = ClassifyPrompt(inputs=inputs)
    
    category = call_bedrock(prompt).strip()
    state['category'] = category
    return {"category": category}

def handle_installation(state: WorkflowState) -> WorkflowState:
    """Handles installation & setup questions"""    
    state['response'] = do_rag(state['question'], InstallationPrompt)
    return state

def handle_security(state: WorkflowState) -> WorkflowState:
    """Handles security & authentication questions"""
    state['response'] = do_rag(state['question'], SecurityPrompt)
    return state

def handle_querying(state: WorkflowState) -> WorkflowState:
    """Handles querying & indexing questions"""
    state['response'] = do_rag(state['question'], QueryPrompt)
    return state

def handle_performance(state: WorkflowState) -> WorkflowState:
    """Handles performance optimization questions"""
    state['response'] = do_rag(state['question'], PerformancePrompt)
    return state

def init_state(question: str) -> WorkflowState:
    """Initialize the workflow state with a question."""
    return WorkflowState(
        question=question,
        category="",
        response=""
    )

### Testing
Because our nodes are individual functions that take in a state dictionary. we can test them individually. Lets test our classifier

In [ ]:
question = "How do I install OpenSearch on AWS?"
state: WorkflowState = init_state(question=question)
classify_question(state)

In [ ]:
question = "How do I install OpenSearch on AWS?"
state: WorkflowState = init_state(question=question)
handle_installation(state)

# Construct Routing Graph
Next lets create and compile our graph. We can use a conditional check to route each question to the correct prompt that's tuned to provide the relevant information from each type of document

In [ ]:
def create_routing_workflow() -> StateGraph:
    """Creates a workflow for routing OpenSearch questions"""
    workflow = StateGraph(WorkflowState)
    
    # Add nodes to our graph
    workflow.add_node("classify", classify_question)
    workflow.add_node("install", handle_installation)
    workflow.add_node("security", handle_security)
    workflow.add_node("query", handle_querying)
    workflow.add_node("performance", handle_performance)
    
    # Create conditional routing
    workflow.add_edge(START, "classify")
    
    # Create conditional edges based on the category
    workflow.add_conditional_edges(
        "classify",
        lambda state: state["category"],
        {
            "INSTALL": "install",
            "SECURITY": "security",
            "QUERY": "query",
            "PERFORMANCE": "performance"
        }
    )
    
    # All handlers lead to END
    workflow.add_edge("install", END)
    workflow.add_edge("security", END)
    workflow.add_edge("query", END)
    workflow.add_edge("performance", END)
    
    # Compile and return the workflow
    return workflow.compile()

graph: StateGraph = create_routing_workflow()

## 2. Testing with Various Questions

Let's test our router with different types of OpenSearch questions:

In [ ]:
# Create our workflow
graph: StateGraph = create_routing_workflow()

# Test questions covering different categories
test_questions = [
    "How do I install OpenSearch on AWS?",
    "What's the best way to implement role-based access control?",
    "How can I write efficient fuzzy match queries?",
    "What's the optimal shard size for large indices?",
    "How do I configure SSL/TLS for cluster security?"
]

print("🔀 Testing our question router...\n")

for question in test_questions:
    print(f"Question: {question}")
    state: WorkflowState = init_state(question=question)
    result = graph.invoke(state)
    print(f"Category: {result['category']}")
    print(f"Response: {result['response']}\n")
    print("-" * 80 + "\n")

## 3. Benefits of the Routing Pattern

Our question routing system provides several advantages:

✅ Specialized handling for each category

✅ More focused and accurate responses

✅ Easy to add new categories

✅ Clear separation of concerns

Next, we'll explore parallel processing to generate multiple solution approaches simultaneously! 🚀